In [1]:
import igl
import scipy as sp
import numpy as np
import meshplot as mp
from meshplot import plot, subplot, interact
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
def adjacency_matrix_to_list(vertices, faces):
    adjacency_list = {i: set() for i in range(len(vertices))}
    for face in faces:
        for i in range(len(face)):
            v1 = face[i]
            v2 = face[(i + 1) % len(face)]
            adjacency_list[v1].add(v2)
            adjacency_list[v2].add(v1)
    for key in adjacency_list:
        adjacency_list[key] = sorted(adjacency_list[key])
    return adjacency_list

In [3]:
def MakeAmatrix(i, V, adjacency_list):
    A = adjacency_list
    #append i in the beginiing of each A[i]
    A = np.insert(A, 0, i)
    for k in A:
        matrix = np.zeros((3,4))
        matrix[0][0] = V[k][0]
        matrix[1][0] = V[k][1]
        matrix[2][0] = V[k][2]
        matrix[1][1] = -V[k][2]
        matrix[2][1] = V[k][1]
        matrix[0][2] = V[k][2]
        matrix[2][2] = -V[k][0]
        matrix[0][3] = -V[k][1]
        matrix[1][3] = V[k][0]
        I = np.eye(3)
        if k == i:
            matrix_fin = np.concatenate((matrix, I), axis=1)
        else:
            matrix = np.concatenate((matrix, I), axis=1)
            matrix_fin = np.concatenate((matrix_fin, matrix), axis=0)  
    matrix_fin_i = np.linalg.inv(np.matmul(matrix_fin.T, matrix_fin))
    vector = np.matmul(matrix_fin_i, matrix_fin.T)
    return vector

In [4]:
def degree_matrix(adjacency_list):
    # Create a diagonal matrix where the diagonal elements are the degrees of each vertex
    degree = []
    for i in adjacency_list:
        degree.append(len(adjacency_list[i]))
    degrees = np.array(degree)
    D = np.diag(degrees)
    return D

In [5]:
def create_adjacency_matrix(vertices, faces):
    num_vertices = vertices.shape[0]
    adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=int)

    # Extract vertex indices for each face
    i, j, k = faces[:, 0], faces[:, 1], faces[:, 2]
    
    # Set adjacency between pairs of vertices for all faces
    adjacency_matrix[i, j] = 1
    adjacency_matrix[j, i] = 1
    adjacency_matrix[i, k] = 1
    adjacency_matrix[k, i] = 1
    adjacency_matrix[j, k] = 1
    adjacency_matrix[k, j] = 1
    
    return adjacency_matrix

In [6]:
def makeitlarge(Ti, matrix, i, adjacency_list_for_i):
    A = np.insert(adjacency_list_for_i, 0, i)  # Insert i at the beginning of the adjacency list
    j=0
    for k in A:
        Ti_block = Ti[:, 3*j:3*(j+1)]
        matrix[3*i:3*(i+1), 3*k:3*(k+1)] = Ti_block
        j += 1    
    return matrix

In [7]:
def makeLbig(L):
    n = L.shape[0]  # Number of vertices
    M1 = np.zeros((3 * n, 3 * n))  # Initialize a (3n, 3n) matrix with zeros
    
    # Fill in the block diagonal
    M1[0::3, 0::3] = L  # First block for x-coordinates
    M1[1::3, 1::3] = L  # Second block for y-coordinates
    M1[2::3, 2::3] = L  # Third block for z-coordinates
    
    return M1

In [8]:
def MakeU(V,handle_points, handle_vertices):
    u = np.zeros(3 * len(V))  # Initialize the flattened vector u
    i = 3 * np.array(handle_points) # Indices for x, y, z components
    k = np.arange(len(handle_points))  # Indices for handle points
    # Fill u with handle vertex coordinates
    u[i] = handle_vertices[k, 0]  # x-coordinates
    u[i + 1] = handle_vertices[k, 1]  # y-coordinates
    u[i + 2] = handle_vertices[k, 2]  # z-coordinates
    return u

In [9]:
def T_bar(vector,Vx_i):
    T1 = Vx_i[0] * vector[0] - Vx_i[1] * vector[3] + Vx_i[2] * vector[2]
    T2 = Vx_i[0] * vector[3] + Vx_i[1] * vector[0] - Vx_i[2] * vector[1]
    T3 = -Vx_i[0] * vector[2] + Vx_i[1] * vector[1] + Vx_i[2] *  vector[0]
    return np.array([T1,T2,T3])

In [10]:
def  Mcon(v,handle_points):
    n = len(v)  # Number of vertices
    M = np.zeros((3 * n, 3 * n))  # Initialize the constraint matrix
    i = np.array(handle_points)
    indices = np.concatenate([3*i, 3*i + 1, 3*i + 2])
    M[indices, indices] = 1
    return M

In [11]:
def Solve(V,F,handle_points,handle_vertices):
    adjacency_list = adjacency_matrix_to_list(V,F)
    adjacency_matrix = create_adjacency_matrix(V,F)
    I = np.eye(len(V))
    D = degree_matrix(adjacency_list)
    L = I - np.matmul(np.linalg.inv(D), adjacency_matrix) #we can make L large such that order of A = order of L
    Vx = np.matmul(L,V[:,0])
    Vy = np.matmul(L,V[:,1])
    Vz = np.matmul(L,V[:,2]) 
    L_big = makeLbig(L) 
    A = np.zeros((3*len(V),3*len(V)))
    for i in range(len(V)):
        Ti = MakeAmatrix(i,V,adjacency_list[i])
        Ti_bar = T_bar(Ti[0:4,:],[Vx[i],Vy[i],Vz[i]])
        A = makeitlarge(Ti_bar, A, i, adjacency_list[i])
    Mcon_1 = Mcon(V,handle_points)
    u = MakeU(V,handle_points, handle_vertices)
    X = A - L_big
    X_f = np.matmul(X.T,X)
    X_f = X_f + Mcon_1
    b = np.matmul(Mcon_1,u)
    v = np.linalg.lstsq(X_f,b, rcond=None)[0]
    x = v.reshape((len(V),3))
    return x

In [12]:
# Define vertices for the diamond
vertices = np.array([
    [0, 0, 1],    # Vertex 0: Top vertex
    [1, 0, 0],    # Vertex 1: Base vertex 1
    [0, 1, 0],    # Vertex 2: Base vertex 2
    [-1, 0, 0],   # Vertex 3: Base vertex 3
    [0, -1, 0],   # Vertex 4: Base vertex 4
    [0, 0, -1]    # Vertex 5: Bottom vertex
])

# Define faces by connecting vertices
faces = np.array([
    [0, 1, 2],    # Face 0
    [0, 2, 3],    # Face 1
    [0, 3, 4],    # Face 2
    [0, 4, 1],    # Face 3
    [5, 1, 4],    # Face 4
    [5, 4, 3],    # Face 5
    [5, 3, 2],    # Face 6
    [5, 2, 1]     # Face 7
])

In [13]:
indices = [2,4]
new_positions = np.array([[0.5,1,0],[0,-1,0]])
V_1 = Solve(vertices,faces, indices, new_positions)

In [14]:
print(vertices)

[[ 0  0  1]
 [ 1  0  0]
 [ 0  1  0]
 [-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]


In [15]:
print(V_1)

[[ 2.50000000e-01 -1.11022302e-15  1.00000000e+00]
 [ 1.25000000e+00 -2.50000000e-01  1.94740061e-15]
 [ 5.00000000e-01  1.00000000e+00  8.67194230e-16]
 [-7.50000000e-01  2.50000000e-01  1.07795713e-15]
 [ 1.11105307e-15 -1.00000000e+00 -1.64900363e-16]
 [ 2.50000000e-01 -1.35749718e-15 -1.00000000e+00]]


In [16]:
plot(vertices, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [17]:
plot(V_1, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.25, 0.0…

In [18]:
#Load the .off file
file_path = '/Users/vrajthakkar/Desktop/IP-2 ARAP/cactus.off'
V , F = igl.read_triangle_mesh(file_path)
plot(V,F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5000501…

In [19]:
def findonering(V,F,i):
    onering = []
    for f in F:
        if i in f:
            for j in f:
                if j != i:
                    onering.append(j)
    return onering

In [20]:
print(V[203])

[0.50276786 0.56758571 0.86119568]


In [49]:
X = findonering(V,F,203)
X = np.append(X,203)
V_f = V[X]
V_f = np.array(V_f + [0,0,0.5])
print(V_f)

[[0.50932294 0.56420672 1.36659473]
 [0.51094753 0.56856835 1.35802537]
 [0.50041866 0.56281751 1.36899501]
 [0.50932294 0.56420672 1.36659473]
 [0.49390334 0.56593752 1.36342192]
 [0.50041866 0.56281751 1.36899501]
 [0.49630097 0.56994456 1.35544348]
 [0.49390334 0.56593752 1.36342192]
 [0.50469297 0.57131249 1.35232574]
 [0.49630097 0.56994456 1.35544348]
 [0.51094753 0.56856835 1.35802537]
 [0.50469297 0.57131249 1.35232574]
 [0.50276786 0.56758571 1.36119568]]


In [50]:
indices = []
new_positions = []
for i in range(len(V)):
    if (V[i][2] < 0.3):
        indices.append(i)
        new_positions.append(np.array([V[i][0],V[i][1],V[i][2]]))
for i in range(len(V_f)):
    new_positions.append(np.array([V_f[i][0],V_f[i][1],V_f[i][2]]))
    indices.append(X[i])
new_vertices = Solve(V,F,indices,np.array(new_positions))
plot(new_vertices, F, shading={"flat": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.6146810…